In [1]:
import numpy as np
import pandas as pd

## MNIST 데이터 불러오기

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_x, batch_y = mnist.train.next_batch(128)

In [4]:
np.shape(batch_x)

(128, 784)

## 필요한 함수들 구현

In [ ]:
def sigmoid(self):
        result = 1.0 / (1.0 + np.exp(-self.x))
        self.back = result*(1-result)
        return result

In [ ]:
def tanh(self):
    result = (np.exp(self.x)-np.exp(-self.x))/(np.exp(self.x)+np.exp(-self.x))
    self.back = 1-result^2
    return result



In [10]:
def relu(x):
    return np.maximum(0,x) 

In [ ]:
def backward_relu(x):
    return np.maximum(0,1)

In [12]:
def softmax(x):
    max = np.max(x,axis=1).reshape(-1,1) # 오버플로우 방지
    temp = np.exp(x-max)
    return temp / np.sum(temp, axis=1).reshape(-1,1)

In [ ]:
def backward_softmax(x):
     return logits-batch_y

In [13]:
def softmax_cross_entropy(logits,y):
    return -np.mean(y*np.log(logits+1e-7))

## 하이퍼 파라미터

In [14]:
lr = 0.1
num_steps = 500
batch_size = 128
display_step = 100
n_hidden_1 = 256 
n_hidden_2 = 256 
num_input = 784 
num_classes = 10

## 3 layers 신경망 구현

In [25]:
np.concatenate((np.ones((1,784)),batch_x)).shape

(129, 784)

In [15]:
class NeuralNet(Activation):

    def __init__(self, x, num_input, n_hidden_1, n_hidden_2, num_classes, y):
        self.params={} # W에 bias 포함
        self.params['W1'] = np.random.randn(num_input+1, n_hidden_1) 
        self.params['W2'] = np.random.randn(n_hidden_1+1, n_hidden_2)
        self.params['W3'] = np.random.randn(n_hidden_2+1, num_classes)
        self.x = np.concatenate((np.ones((1,784)),x))
        self.y = y
#        self.back = 1
    
    def forward(self):
        W1,W2,W3 = self.params['W1'],self.params['W2'],self.params['W3']
        
        self.layer_1 = relu(np.matmul(self.x, W1))
        self.layer_2 = relu(np.matmul(self.layer_1, W2))
        self.output = softmax(np.matmul(self.layer_2, W3))l

        return self.output
    
    def backward(self, epochs, lr):
        W1,W2,W3 = self.params['W1'],self.params['W2'],self.params['W3']

        for k in range(epochs):
#            for i in range(self.x.shape[0]):
            dW3 = np.multiply(self.softmax(np.matmul(self.layer_2,W3)).back, y-W3) 
            dW2 = np.multiply(self.relu(np.matmul(self.layer_1,W2)).back, np.dot(W2, dW3))
            dW1 = np.multiply(self.relu(np.matmul(self.x,W1)).back, np.dot(W1, dW2))                
                
            self.params["W1"] = -lr*dW1
            self.params["W2"] = -lr*dW2
            self.params["W3"] = -lr*dW3
                
      
    
    def relu(self):
        self.back = np.maximum(0,1)
        return np.maximum(self.x,0)
 
    def sigmoid(self):
        k = 1.0 / (1.0 + np.exp(-self.x))
        self.back = k*(1-k)
        return k
       
    def tanh(self):
        k = (np.exp(self.x)-np.exp(-self.x))/(np.exp(self.x)+np.exp(-self.x))
        self.back = 1-(k**2)
        return k
    
    def softmax(x):
        max = np.max(x,axis=1).reshape(-1,1) # 오버플로우 방지
        temp = np.exp(x-max)
        k = temp / np.sum(temp, axis=1).reshape(-1,1)
        self.back = 1-k
        return k
    
    def loss(self):
        logits = self.output #self.forward(self.x)
        return softmax_cross_entropy(logits, self.y)
        
    def accuracy(self):
        yhat = self.output #self.forward(self.x)
        yhat = np.argmax(yhat, axis=1)
        y = np.argmax(self.y, axis=1)
        return np.sum(y==yhat)/float(self.x.shape[0])
        


In [16]:
net = NeuralNet(num_input, n_hidden_1, n_hidden_2, num_classes)

In [17]:
net.predict(batch_x)

array([[  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   1.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   1.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   4.17315385e-311,   7.63049099e-050, ...,
          0.00000000e+000,   1.10926040e-176,   0.00000000e+000],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   1.86221425e-108, ...,
          0.00000000e+000,   3.89830047e-051,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,   1.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   3.53701596e-320,   0.00000000e+000, ...,
          0.00000000e+000,   1.00000000e+000,   0.00000000e+000]])

In [18]:
net.loss(batch_x, batch_y)

1.4355178925223713

In [380]:
W1.size

200704

In [ ]:
class Activation:

    def __init__(self, x):
        self.x = x
        self.back = 1

    def sigmoid(self):
        output = 1.0 / (1.0 + np.exp(-self.x))
        self.back = output*(1-output)
        return output
    
    def relu(self):
        self.back = np.maximum(0,1)
        return np.maximum(self.x,0)
    
    def tanh(self):
        output = (np.exp(self.x)-np.exp(-self.x))/(np.exp(self.x)+np.exp(-self.x))
        self.back = 1-(output**2)
        return output
    
    def softmax(x):
        max = np.max(x,axis=1).reshape(-1,1) # 오버플로우 방지
        temp = np.exp(x-max)
        output = temp / np.sum(temp, axis=1).reshape(-1,1)
        self.back = 1-output
        return output

In [341]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

In [21]:
np.zeros_like(10)

array(0)

In [ ]:
class NeuralNetwork(object):

	def __init__(self, inputs, hidden, outputs, activation='tanh', output_act='softmax'):
		
		# Hidden layer activation function
		if activation == 'sigmoid':
			self.activation = sigmoid
			self.activation_prime = sigmoid_prime
		elif activation == 'tanh':
			self.activation = tanh
			self.activation_prime = tanh_prime
		elif activation == 'linear':
			self.activation = linear
			self.activation_prime = linear_prime

		# Output layer activation function
		if output_act == 'sigmoid':
			self.output_act = sigmoid
			self.output_act_prime = sigmoid_prime
		elif output_act == 'tanh':
			self.output_act = tanh
			self.output_act_prime = tanh_prime
		elif output_act == 'linear':
			self.output_act = linear
			self.output_act_prime = linear_prime
		elif output_act == 'softmax':
			self.output_act = softmax
			self.output_act_prime = linear_prime

		# Weights initializarion
		self.wi = np.random.randn(inputs, hidden)/np.sqrt(inputs)
		self.wo = np.random.randn(hidden + 1, outputs)/np.sqrt(hidden)

		# Weights updates initialization
		self.updatei = 0
		self.updateo = 0


	def feedforward(self, X):

		# Hidden layer activation
		ah = self.activation(np.dot(X, self.wi))
			
		# Adding bias to the hidden layer results
		ah = np.concatenate((np.ones(1).T, np.array(ah)))

		# Outputs
		y = self.output_act(np.dot(ah, self.wo))

		# Return the results
		return y


	def fit(self, X, y, epochs=10, learning_rate=0.2, learning_rate_decay = 0 , momentum = 0, verbose = 0):
		
		# Timer start
		startTime = time.time()

		# Epochs loop
		for k in range(epochs):
	
			# Dataset loop
			for i in range(X.shape[0]):

				# Hidden layer activation
				ah = self.activation(np.dot(X[i], self.wi))
			
				# Adding bias to the hidden layer
				ah = np.concatenate((np.ones(1).T, np.array(ah))) 

				# Output activation
				ao = self.output_act(np.dot(ah, self.wo))

				# Deltas	
				deltao = np.multiply(self.output_act_prime(ao),y[i] - ao)
				deltai = np.multiply(self.activation_prime(ah),np.dot(self.wo, deltao))

				# Weights update with momentum
				self.updateo = momentum*self.updateo + np.multiply(learning_rate, np.outer(ah,deltao))
				self.updatei = momentum*self.updatei + np.multiply(learning_rate, np.outer(X[i],deltai[1:]))

				# Weights update
				self.wo += self.updateo
				self.wi += self.updatei

			# Print training status
			if verbose == 1:
				print 'EPOCH: {0:4d}/{1:4d}\t\tLearning rate: {2:4f}\t\tElapse time [seconds]: {3:5f}'.format(k,epochs,learning_rate, time.time() - startTime)
				
			# Learning rate update
			learning_rate = learning_rate * (1 - learning_rate_decay)

	def predict(self, X): 

		# Allocate memory for the outputs
		y = np.zeros([X.shape[0],self.wo.shape[1]])

		# Loop the inputs
		for i in range(0,X.shape[0]):

			y[i] = self.feedforward(X[i])

		# Return the results
		return y


# Activation functions
def sigmoid(x):
	return 1.0/(1.0 + np.exp(-x))

def sigmoid_prime(x):
	return sigmoid(x)*(1.0-sigmoid(x))

def tanh(x):
	return np.tanh(x)

def tanh_prime(x):
	return 1.0 - x**2

def softmax(x):
    return (np.exp(np.array(x)) / np.sum(np.exp(np.array(x))))

def softmax_prime(x):
    return softmax(x)*(1.0-softmax(x))

def linear(x):
	return x

def linear_prime(x):
	return 1